# 🧭 Projet - Problème du Voyageur de Commerce (TSP)

---

Ce projet explore des heuristiques pour résoudre le **Problème du Voyageur de Commerce (Travelling Salesman Problem)**.  
Nous allons implémenter et comparer différentes approches pour trouver un circuit de coût minimal visitant chaque ville une seule fois.

---

👨‍💻 Myriam HADDOUK & Myriam SCHULMANN  
🎓 Projet Mathématiques-informatique  
📅 Juin 2025  
